In [1]:
import functools

import numpy as np
from scipy.special import expit

import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql import Window
import pyspark.sql as pysql
from pyspark.sql.types import StringType, IntegerType, MapType, ArrayType, FloatType, BooleanType
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from operator import add
from functools import reduce

from scipy.special import expit

pd.set_option('display.max_columns', None)
spark.conf.set('spark.sql.session.timeZone', 'UTC')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
133,application_1652123621130_0135,pyspark,idle,Link,Link,gakumar,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
site_2m = ['094809dc-508c-4cb4-a8b8-dfa43c831b6d','5a031aca-77ca-4ad5-bdd5-c6f1357be883','d2647e1c-3bfe-417c-8875-a7fdf96b3512','92b26c34-5495-4bb0-acce-a83b1948cc46','584c607b-f233-4bfc-8065-9f3547063c11','d78b29a2-8aad-44d6-b44d-d8a92c6f6a62']
site_3m = ['7100bfe0-231e-4362-9eac-f8b525172a7c','d7b217c8-61fc-4a25-ac25-986af0ef1baf','e00a9b82-7fae-4e3b-9cb7-5a5f2ac5da17','ce9a1523-f9f8-45a3-963e-c387d01d0263','a74ec2d2-fbcd-4b2f-b063-83d06bd3d0cb','752e66ae-3aab-4ff8-85c2-44cd88bd2baf','ecdf8571-1b9b-42e9-bac0-2cfefe229d4c','52e3823e-6991-40fb-9593-aad6ac0435c4','70dd4f8e-63db-4987-94c9-4c734aa0d359','506269ba-66cd-4fc7-bb29-738a89320ae9','b1e52686-5056-4e04-ac9d-418f49a59a13','1724c494-5e7b-495c-8403-1d866df79a30','9ebe4475-c11e-4332-870b-f34175284368','4fdfe1a4-f033-4e99-97f0-56e7522cd1d9','61b4bf36-7872-4dcc-98e4-014094aa355d','00b6135e-fdee-4916-b48d-b2a81fe8ddc5','87d96b59-dfd0-4f5c-be3f-4f27f03e3a1e']
site_4m = ['e006ba19-6a36-4eac-856c-8137e938dd7d','becdf2ec-cd2a-44e3-b9bb-bba05a2c39e2','c953b45d-c36e-4be7-a7f6-50945532193b','32b784f3-2227-4151-a6c9-e86af3b27866','3638f1ba-99de-4046-b078-ca7ae961bd0e','e745b10b-31a0-4cd4-bd28-8a95ef101401','e5ba9726-1709-4567-934b-dc6d49eaafbc','99aab171-145c-4206-a21c-2748dd0ad0d0','7322aa78-d7cc-4e3c-ad89-b39eaf379126','24395158-7f3a-446b-bbc6-86133cddb090','01442017-e218-47fa-a3d5-f555e614b5cd','4cb62d8d-f6e5-441d-aced-22ee0b55e513','90573ddc-fdda-4127-bbff-5fbfbe9db6d8','22316bd0-0a39-4366-b02d-2389d0f6c6be','079220c2-880e-4073-b9d0-7b1db6e67ef5','0783acd5-32b2-45f0-87c4-1a4cb1738b46','761d1b5d-15c3-4e95-a068-124ed11054c2','b05e7644-c669-412f-a75f-beb58138e550','e36d6e30-dca4-45de-b680-b2bfbf6e818b','b1d890da-52fc-4ea4-96a5-4a4b745ef69a','29fcd25d-3769-4958-b991-e3f8ba310a40','308ea5b7-61a6-4dde-84df-2f1a744add88']
site_6m = ['6ce5881d-ca21-4bd8-bcec-08d75946f90f','c0a26f80-d10b-4112-af4c-7a3ef79514f3','a99544cd-1a9f-4528-b071-31c552b6764c']
site_7m = ['ab878347-19af-4e9d-ba50-f84a4395afab','b8cc6926-26df-4a7e-a27d-7f44162b73ef']
site_7m_pt2 = ['a18ff2f9-d1d9-45d9-afc1-a14dcbe6e52c','1db53bf9-b69e-4955-841e-5d18564f3ffd','fe45458d-e425-4e65-85b6-1890bcb7ad70','35686165-8ee5-4ad9-a14f-09a720ba5ee3','3afe69fc-7189-4505-b00a-fe5922765dba','5a86c5bb-2aab-4c7d-a765-e6001033ff0c','9d4ef64d-4efd-43db-8d08-be8f62a9e941','01b02012-ef0e-4607-a627-6f023241ad6e','9c4c6fe6-32e6-4041-b756-ad774f57fdf7']

# site_list = site_2m + site_3m + site_4m
# site_list = site_6m + site_7m + site_7m_pt2

site_list = site_2m + site_3m + site_4m + site_6m + site_7m + site_7m_pt2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
date = '2022-03-06'
env = 'production'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def read_hourly_data(s3_path, hours):
    final_df = None
    for hr in hours:
        print('reading data for hr: {0}'.format(hr))
        _s3_path = s3_path.format(hr=hr)
        print('reading from s3')
        df = spark.read.parquet(_s3_path)
        print(df.count())
        if final_df is None:
            final_df = df
        else:
            _col1 = set(final_df.columns)
            _col2 = set(df.columns)
            for c in list(_col1.difference(_col2)):
                df = df.withColumn(c, F.lit(''))
            for c in list(_col2.difference(_col1)):
                final_df = final_df.withColumn(c, F.lit(''))
            final_df = final_df.unionByName(df)
    print('total records: {0}'.format(final_df.count()))
    return final_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# df_label.filter(F.col("name").startswith("ae")).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# (526, 526)
# df_label.select("switch_id", "name").distinct().count(), df_label.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# (481, 481)
# df_hrs.select(*sel_cols).distinct().count(), df_hrs.select("switch_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def predict_prob(df, coefficients, intercept, threshold):
    features = []
    coef = []
    for k, v in coefficients.items():
        features.append(k)
        coef.append(float(v))
    df = df.withColumn('uplink_prob_score', F.when(F.col('events_count') >= 10,
                                                   predict_uplink_udf(F.struct(*features),
                                                                      F.array([F.lit(x) for x in coef]),
                                                                      F.lit(intercept))
                                                   ).otherwise(0.0)).withColumn(
        'uplink_value_threshold', F.when(F.col('uplink_prob_score') >= threshold, 1).otherwise(0))
    return df


@udf(returnType=IntegerType())
def phy_count_udf(interfaces, device_type):
    return len(
        [interface for interface in interfaces if
         get_port_type(interface['name'], interface['media_type'], device_type) == 'phy'])


@udf(returnType=IntegerType())
def active_phy_count_udf(interfaces, device_type):
    return len([interface for interface in interfaces if
                get_port_type(interface['name'], interface['media_type'], device_type) == 'phy' and is_config_port(
                    interface, device_type)])


def get_port_type(interface_name, media_type, device_type):
    name = interface_name if interface_name is not None else ''
    media = media_type if media_type is not None else ''
    device_type = device_type if device_type is not None else ''
    if device_type == 2:
        media = 'copper'
    if media == 'copper' or media == 'fiber':
        return 'phy' if len(name) >= 3 and (name[:2] == 'ge') | (name[:2] == 'xe') | (name[:2] == 'et') | (
                name[:3] == 'mge') else 'vir'
    else:
        return 'vir'


def is_config_port(interface, device_type):
    # vlan_ids = interface['vlan_ids'] if interface['vlan_ids'] is not None else []
    # device_type = device_type if device_type is not None else ''
    # vlan_flag = len(vlan_ids) > 0
    # if device_type == 2:
    #     vlan_flag = True
    return (interface['link']) & (interface['admin_status']) & (not interface['unconfigured'])  #& vlan_flag


@udf(returnType=StringType())
def vp_udf(name, media_type, device_type):
    return get_port_type(name, media_type, device_type)


@udf(returnType=IntegerType())
def remotecap_udf(x):
    if (x == 'Bridge Router') or (x == 'Bridge'):
        return 1
    else:
        return 0


@udf(returnType=IntegerType())
def mbps_udf(x):
    if x > 1000:
        return 1
    else:
        return 0


@udf(returnType=IntegerType())
def remotehw_udf(x):
    if x is not None and len(x) >= 2 and x[:2] == 'AP':
        return 0
    else:
        return 1


@udf(returnType=IntegerType())
def bridge_udf(stpbridge_root_port, name):
    if stpbridge_root_port == name:
        return 1
    else:
        return 0


@udf(returnType=FloatType())
def get_rank_pct_udf(rank, event_count):
    return len([i for i in rank if i == 1]) / event_count


@udf(returnType=IntegerType())
def reix_udf(name):
    try:
        ix = name.split('-')[1].split('/')[0]
        if ix.isdigit():
            return int(ix)
    except IndexError:
        return 0
    return 0


@udf(returnType=IntegerType())
def is_root_udf(current, current_other, all_ports):
    if current == 0:
        return 0
    _all_others = all_ports
    if current_other > 0:
        _all_others = [t for t in all_ports if t != current_other]
    _max = 0
    if len(_all_others) > 0:
        _max = max(_all_others)
    if current > 0 and current >= _max:
        return 1
    else:
        return 0


@udf(returnType=FloatType())
def predict_uplink_udf(feature_values, coefficients, intercept):
    coef = np.array(coefficients)
    sigmoid_val = expit(np.dot(feature_values, coef.T) + intercept).ravel()
    return float(sigmoid_val[0])


@udf(returnType=FloatType())
def arrays_weighed_avg(avg_arr, events_arr):
    avg_arr = np.array(avg_arr)
    events_arr = np.array(events_arr)
    return float(np.sum(avg_arr * events_arr) / np.sum(events_arr))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def add_switch_level_features(df):
    df = df.withColumn('phy_ports_count', phy_count_udf(F.col('interfaces'), F.col('device_type'))) \
        .withColumn('active_phy_ports_count', active_phy_count_udf(F.col('interfaces'), F.col('device_type')))
    df = df.where('active_phy_ports_count>=1').withColumn('when', F.col('when') / 1000000)
    return df


def filter_zero_counters_records(df):
    df = df.withColumn('max_tx', F.array_max('interfaces.tx_packets')).withColumn(
        'max_rx', F.array_max('interfaces.rx_packets')).where('max_tx>0 or max_rx>0').drop('max_tx', 'max_rx')
    df = df.filter((F.col('delta')) & (F.col('delta_interval') > 0) & (F.col('switch_id') != "")
                   & (~F.col('model').startswith("SSR")))
    return df


def generate_interface_data(df):
    fields = ['switch_id', 'org_id', 'site_id', 'when', 'model', 'device_type', 'stpbridge_root_port']
    dfin = df.select(*fields, 'phy_ports_count', 'active_phy_ports_count', F.explode('interfaces').alias('interfaces')) \
        .select(*fields, 'phy_ports_count', 'active_phy_ports_count', F.col('interfaces.name').alias('name'),
                F.col('interfaces.link').alias('link'),
                F.col('interfaces.mbps').alias('mbps'),
                F.col('interfaces.admin_status').alias('admin_status'),
                F.col('interfaces.media_type').alias('media_type'),
                F.col('interfaces.remote_system_capability').alias('remote_system_capability'),
                F.col('interfaces.remote_hardware').alias('remote_hardware'),
                F.col('interfaces.vlan_ids').alias('vlan_ids'),
                F.col('interfaces.rx_packets').alias('rx_packets'),
                F.col('interfaces.tx_packets').alias('tx_packets'),
                F.col('interfaces.rx_mcast_packets').alias('rx_mcast_packets'),
                F.col('interfaces.tx_mcast_packets').alias('tx_mcast_packets'),
                F.col('interfaces.rx_bcast_packets').alias('rx_bcast_packets'),
                F.col('interfaces.tx_bcast_packets').alias('tx_bcast_packets'),
                F.col('interfaces.tx_ucast_packets').alias('tx_ucast_packets'),
                F.col('interfaces.rx_ucast_packets').alias('rx_ucast_packets'))
    dfin = dfin.withColumn('type', vp_udf(F.col('name'), F.col('media_type'), F.col('device_type')))
    dfin = dfin.filter((F.col('link')))
    dfin = dfin.filter(dfin.type == 'phy')
    dfin = dfin.withColumn('re_ix', reix_udf(dfin.name))
    return dfin


def add_derived_features(df):
    df_uplink = df.withColumn('remotecap', remotecap_udf(F.col('remote_system_capability'))) \
        .withColumn('rmbps', mbps_udf(F.col('mbps'))) \
        .withColumn('remotehw', remotehw_udf('remote_hardware')) \
        .withColumn('num_vlan', F.size('vlan_ids')) \
        .withColumn('bridge', bridge_udf(F.col('stpbridge_root_port'), F.col('name')))

    df_uplink = df_uplink.where('remotehw==1')  # Filter ports connected to AP
    df_uplink = add_record_level_features(df_uplink)
    return df_uplink


def add_record_level_features(df_uplink):
    w = Window.partitionBy(F.col('switch_id'), F.col('when'))
    df_uplink = df_uplink.withColumn(
        "rxpkt_rank",
        F.when(F.col('rx_packets') > 0, F.dense_rank().over(w.orderBy(F.col('rx_packets').desc()))).otherwise(
            100)).withColumn(
        "txpkt_rank",
        F.when(F.col('tx_packets') > 0, F.dense_rank().over(w.orderBy(F.col('tx_packets').desc()))).otherwise(
            100)).withColumn(
        "rx_upkt_rank", F.when(F.col('rx_ucast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('rx_ucast_packets').desc()))).otherwise(
            100)).withColumn(
        "tx_upkt_rank", F.when(F.col('tx_ucast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('tx_ucast_packets').desc()))).otherwise(
            100)).withColumn(
        "rx_bpkt_rank", F.when(F.col('rx_bcast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('rx_bcast_packets').desc()))).otherwise(
            100)).withColumn(
        "tx_bpkt_rank", F.when(F.col('tx_bcast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('tx_bcast_packets').desc()))).otherwise(
            100)).withColumn(
        "rx_mpkt_rank", F.when(F.col('rx_mcast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('rx_mcast_packets').desc()))).otherwise(
            100)).withColumn(
        "tx_mpkt_rank", F.when(F.col('tx_mcast_packets') > 0,
                               F.dense_rank().over(w.orderBy(F.col('tx_mcast_packets').desc()))).otherwise(
            100)).withColumn(
        "physical_port_count", F.count(F.col('switch_id')).over(w)).withColumn(
        "bridge_port_count", F.sum(F.col('remotecap')).over(w)).withColumn(
        'rx_packets_all', F.collect_list('rx_packets').over(w)).withColumn(
        'tx_packets_all', F.collect_list('tx_packets').over(w)).withColumn(
        'rx_ucast_packets_all', F.collect_list('rx_ucast_packets').over(w)).withColumn(
        'tx_ucast_packets_all', F.collect_list('tx_ucast_packets').over(w)).withColumn(
        'rx_bcast_packets_all', F.collect_list('rx_bcast_packets').over(w)).withColumn(
        'tx_bcast_packets_all', F.collect_list('tx_bcast_packets').over(w)).withColumn(
        'rx_mcast_packets_all', F.collect_list('rx_mcast_packets').over(w)).withColumn(
        'tx_mcast_packets_all', F.collect_list('tx_mcast_packets').over(w)).withColumn(
        'rx_root', is_root_udf('rx_packets', 'tx_packets', 'tx_packets_all')).withColumn(
        'tx_root', is_root_udf('tx_packets', 'rx_packets', 'rx_packets_all')).withColumn(
        'rx_ucast_root', is_root_udf('rx_ucast_packets', 'tx_ucast_packets', 'tx_ucast_packets_all')).withColumn(
        'tx_ucast_root', is_root_udf('tx_ucast_packets', 'rx_ucast_packets', 'rx_ucast_packets_all')).withColumn(
        'rx_bcast_root', is_root_udf('rx_bcast_packets', 'tx_bcast_packets', 'tx_bcast_packets_all')).withColumn(
        'tx_bcast_root', is_root_udf('tx_bcast_packets', 'rx_bcast_packets', 'rx_bcast_packets_all')).withColumn(
        'rx_mcast_root', is_root_udf('rx_mcast_packets', 'tx_mcast_packets', 'tx_mcast_packets_all')).withColumn(
        'tx_mcast_root', is_root_udf('tx_mcast_packets', 'rx_mcast_packets', 'rx_mcast_packets_all')).withColumn(
        'tx_rx_root', F.col('rx_root') * F.col('tx_root')).withColumn(
        'tx_rx_ucast_root', F.col('rx_ucast_root') * F.col('tx_ucast_root')).withColumn(
        'tx_rx_bcast_root', F.col('rx_bcast_root') * F.col('tx_bcast_root')).withColumn(
        'tx_rx_mcast_root', F.col('rx_mcast_root') * F.col('tx_mcast_root'))

    return df_uplink


def derive_rank_features(df):
    df_uplink = df.groupBy('org_id', 'site_id', 'switch_id', 'device_type', 'name').agg(
        F.first('re_ix').alias('re_ix'),
#         F.first('ae_port').alias('ae_port'),
        F.count('switch_id').alias('events_count'),
        F.max('remotecap').alias('remotecap'),
        F.max('rmbps').alias('rmbps'),
        F.max('bridge').alias('bridge'),
        F.max('num_vlan').alias('num_vlan'),
        F.sum('rx_packets').alias('rx_packets'),
        F.sum('tx_packets').alias('tx_packets'),
        F.sum('rx_bcast_packets').alias('rx_bcast_packets'),
        F.sum('tx_bcast_packets').alias('tx_bcast_packets'),
        F.sum('rx_mcast_packets').alias('rx_mcast_packets'),
        F.sum('tx_mcast_packets').alias('tx_mcast_packets'),
        F.sum('rx_ucast_packets').alias('rx_ucast_packets'),
        F.sum('tx_ucast_packets').alias('tx_ucast_packets'),
        F.collect_list('rxpkt_rank').alias("rxpkt_ranks"),
        F.collect_list('txpkt_rank').alias("txpkt_ranks"),
        F.collect_list('rx_upkt_rank').alias("rx_upkt_ranks"),
        F.collect_list('tx_upkt_rank').alias("tx_upkt_ranks"),
        F.collect_list('rx_bpkt_rank').alias("rx_bpkt_ranks"),
        F.collect_list('tx_bpkt_rank').alias("tx_bpkt_ranks"),
        F.collect_list('rx_mpkt_rank').alias("rx_mpkt_ranks"),
        F.collect_list('tx_mpkt_rank').alias("tx_mpkt_ranks"),
        F.avg('rxpkt_rank').alias("rxpkt_rank_avg"),
        F.avg('txpkt_rank').alias("txpkt_rank_avg"),
        F.avg('rx_upkt_rank').alias("rx_upkt_rank_avg"),
        F.avg('tx_upkt_rank').alias("tx_upkt_rank_avg"),
        F.avg('rx_bpkt_rank').alias("rx_bpkt_rank_avg"),
        F.avg('tx_bpkt_rank').alias("tx_bpkt_rank_avg"),
        F.avg('rx_mpkt_rank').alias("rx_mpkt_rank_avg"),
        F.avg('tx_mpkt_rank').alias("tx_mpkt_rank_avg"),
        F.avg('physical_port_count').alias("physical_port_count"),
        F.avg('bridge_port_count').alias("bridge_port_count"),
        F.avg('rx_root').alias("rx_root_avg"),
        F.avg('tx_root').alias("tx_root_avg"),
        F.avg('tx_rx_root').alias("tx_rx_root_avg"),
        F.avg('rx_ucast_root').alias("rx_ucast_root_avg"),
        F.avg('tx_ucast_root').alias("tx_ucast_root_avg"),
        F.avg('tx_rx_ucast_root').alias("tx_rx_ucast_root_avg"),
        F.avg('rx_bcast_root').alias("rx_bcast_root_avg"),
        F.avg('tx_bcast_root').alias("tx_bcast_root_avg"),
        F.avg('tx_rx_bcast_root').alias("tx_rx_bcast_root_avg"),
        F.avg('rx_mcast_root').alias("rx_mcast_root_avg"),
        F.avg('tx_mcast_root').alias("tx_mcast_root_avg"),
        F.avg('tx_rx_mcast_root').alias("tx_rx_mcast_root_avg")).withColumn(
        'rxpkt_rank_pct', get_rank_pct_udf(F.col('rxpkt_ranks'), F.col('events_count'))).withColumn(
        'txpkt_rank_pct', get_rank_pct_udf(F.col('txpkt_ranks'), F.col('events_count'))).withColumn(
        'rx_upkt_rank_pct', get_rank_pct_udf(F.col('rx_upkt_ranks'), F.col('events_count'))).withColumn(
        'tx_upkt_rank_pct', get_rank_pct_udf(F.col('tx_upkt_ranks'), F.col('events_count'))).withColumn(
        'rx_bpkt_rank_pct', get_rank_pct_udf(F.col('rx_bpkt_ranks'), F.col('events_count'))).withColumn(
        'tx_bpkt_rank_pct', get_rank_pct_udf(F.col('tx_bpkt_ranks'), F.col('events_count'))).withColumn(
        'rx_mpkt_rank_pct', get_rank_pct_udf(F.col('rx_mpkt_ranks'), F.col('events_count'))).withColumn(
        'tx_mpkt_rank_pct', get_rank_pct_udf(F.col('tx_mpkt_ranks'), F.col('events_count')))

    return df_uplink

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# def update_ae_port(df_hrs, df_uplink):
#     df_ae = df_hrs.select("switch_id", "when", F.explode("lldpneighbors").alias("l")) \
#         .select("switch_id", "when", "l.local_port_id", "l.local_parent_iface_name")
#     df_ae = df_ae.filter(F.col("local_parent_iface_name").startswith("ae")) \
#         .withColumn("ae_port", F.lit(1)) \
#         .withColumnRenamed("local_port_id", "name")
    
#     df_uplink = df_uplink.join(df_ae, ["switch_id", "when", "name"], how="left")
    
#     df_ae_uplink = df_uplink.filter(F.col("ae_port")==1).groupBy(["switch_id", "when", "local_parent_iface_name"]).agg(
#         F.sum('rx_packets').alias('rx_packets_ae'),
#         F.sum('tx_packets').alias('tx_packets_ae'),
#         F.sum('rx_mcast_packets').alias('rx_mcast_packets_ae'),
#         F.sum('tx_mcast_packets').alias('tx_mcast_packets_ae'),
#         F.sum('rx_bcast_packets').alias('rx_bcast_packets_ae'),
#         F.sum('tx_bcast_packets').alias('tx_bcast_packets_ae'),
#         F.sum('rx_ucast_packets').alias('rx_ucast_packets_ae'),
#         F.sum('tx_ucast_packets').alias('tx_ucast_packets_ae'),
#     )
    
#     df_uplink = df_uplink.join(df_ae_uplink, ["switch_id", "when", "local_parent_iface_name"], how="left")
    
#     data_cols = ['rx_packets', 'tx_packets', 'rx_mcast_packets', 'tx_mcast_packets', 'rx_bcast_packets', 
#                  'tx_bcast_packets', 'tx_ucast_packets', 'rx_ucast_packets']
#     for col in data_cols:
#         df_uplink = df_uplink.withColumn(col, F.when(F.col("ae_port")==1, F.col(col+"_ae")).otherwise(F.col(col))).drop(col+"_ae")
        
#     return df_uplink

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def get_hr_str(x):
    x_str = str(x)
    if x<10:
        x_str = "0"+x_str
    return x_str

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
for i in range(21, 24):
    hr = get_hr_str(i)
    hr_1 = get_hr_str(i-1)
    hr_2 = get_hr_str(i-2)
    
    hr_val = i
    hours = [hr_2, hr_1, hr]
    print("Check hr: ", hr)
    print("Check hr_val: ", hr_val)
    print("Check hours: ", hours)
    
    oc_stats_s3_path = 's3://mist-secorapp-'+env+'/oc-stats-analytics/oc-stats-analytics-'+env+'/dt='+date+'/hr={hr}/*'
    df_hrs = read_hourly_data(oc_stats_s3_path, hours)
    
    df_hrs = df_hrs.filter(F.col("site_id").isin(site_list)).persist()

    sel_cols = ['switch_id', 'uplink_combined']
    df_label = df_hrs.select(*sel_cols).distinct() \
        .select('switch_id', F.explode('uplink_combined')) \
        .withColumnRenamed("col", "name") \
        .withColumn("label", F.lit(1))
    
    df_hrs = filter_zero_counters_records(df_hrs)
    df_hrs = add_switch_level_features(df_hrs)
    df_uplink = generate_interface_data(df_hrs)
    
#     df_uplink = update_ae_port(df_hrs, df_uplink)
    df_uplink = add_derived_features(df_uplink)
    df_uplink = derive_rank_features(df_uplink)
    df_uplink = df_uplink.join(df_label, ["switch_id", "name"], how="left")
    df_uplink = df_uplink.na.fill(value=0)
    df_uplink = df_uplink.withColumn("hour", F.lit(hr_val)).persist()
    
    # Saving at local_path 
    local_s3_path = "s3://mist-data-science-dev/gakumar/uplink/training_data/old_logic_all_vars_data_12may/hr="+str(hr)+"/"

    print(local_s3_path)
    df_uplink.repartition(1).write.parquet(local_s3_path, mode='overwrite')
    
    print("########################################################################################")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check hr:  21
Check hr_val:  21
Check hours:  ['19', '20', '21']
reading data for hr: 19
reading from s3
264821
reading data for hr: 20
reading from s3
264682
reading data for hr: 21
reading from s3
264942
total records: 794445
s3://mist-data-science-dev/gakumar/uplink/training_data/old_logic_all_vars_data_12may/hr=21/
########################################################################################
Check hr:  22
Check hr_val:  22
Check hours:  ['20', '21', '22']
reading data for hr: 20
reading from s3
264682
reading data for hr: 21
reading from s3
264942
reading data for hr: 22
reading from s3
265042
total records: 794666
s3://mist-data-science-dev/gakumar/uplink/training_data/old_logic_all_vars_data_12may/hr=22/
########################################################################################
Check hr:  23
Check hr_val:  23
Check hours:  ['21', '22', '23']
reading data for hr: 21
reading from s3
264942
reading data for hr: 22
reading from s3
265042
reading data for hr: 